## Project Overview

This project generates Seinfeld TV scripts using Recurrent Neural Networks (RNNs). The model is trained on a dataset containing scripts from 9 seasons of Seinfeld. The neural network learns patterns from the training data to generate new, "fake" TV scripts that mimic the style and dialogue of the original show.

## 2. Data Loading
The Seinfeld scripts are loaded from a text file, and the first 500 characters of the text are printed to verify that the data has been loaded correctly.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
import importlib

# Defining project directory in Drive
project_dir = "/content/drive/My Drive/projects/tv-script-gen/"

# Adding the project directory to sys.path to make sure both helper.py and problem_unittests.py can be imported
sys.path.append(project_dir)

# Force reload the helper module and problem_unittests module
import helper
importlib.reload(helper)

import problem_unittests
importlib.reload(problem_unittests)

# Load data
data_dir = os.path.join(project_dir, "data", "seinfeld_scripts.txt")
text = helper.load_data(data_dir)

# Printing first 500 characters to verify
print(text[:500])

Mounted at /content/drive
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he d


## 3. Data Exploration
Exploreing the dataset by providing statistics about the text, such as the number of unique words, the total number of lines, and the average number of words per line. A sample of the first few lines of dialogue is also displayed to give an idea of the content and structure of the scripts.

In [3]:
view_line_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y


## 4. Data Pre-processing
pre-processing functions the entire dataset using the defined functions. The `create_lookup_tables` function creates dictionaries to map words to unique integer IDs and vice versa. The `token_lookup` function generates a dictionary to replace punctuation with specific tokens, ensuring that punctuation does not interfere with word identification.

In [4]:
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}
    
    return (vocab_to_int, int_to_vocab)


tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


In [5]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    
    tokens = {
        '.': '||Period||',
        ',': '||Comma||',
        '"': '||Quotation_Mark||',
        ';': '||Semicolon||',
        '!': '||Exclamation_mark||',
        '?': '||Question_mark||',
        '(': '||Left_Parentheses||',
        ')': '||Right_Parentheses||',
        '-': '||Dash||',
        '\n': '||Return||'
    }

    return tokens

tests.test_tokenize(token_lookup)

Tests Passed


In [6]:
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

In [7]:
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## 5. Neural Network Construction
The RNN architecture is defined using PyTorch. The RNN class is implemented with methods for initialization, forward propagation, and hidden state initialization. The model is designed to process sequences of words and predict the next word in the sequence.

In [8]:
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

In [9]:
from torch.utils.data import TensorDataset, DataLoader

def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    
    n_batches = len(words)//batch_size

    words = words[:n_batches*batch_size]

    x, y = [], []
    for idx in range(0, len(words) - sequence_length):
        idx_stop = sequence_length + idx
        batch_x = words[idx: idx_stop]
        x.append(batch_x)
        batch_y = words[idx_stop]
        y.append(batch_y)

    data = TensorDataset(torch.from_numpy(np.asarray(x)), torch.from_numpy(np.asarray(y)))
    data_loader = DataLoader(data, shuffle=False, batch_size=batch_size)
    
    return data_loader

In [10]:
# Test dataloader
test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = next(data_iter)  # ✅ Corrected

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 0,  1,  2,  3,  4],
        [ 1,  2,  3,  4,  5],
        [ 2,  3,  4,  5,  6],
        [ 3,  4,  5,  6,  7],
        [ 4,  5,  6,  7,  8],
        [ 5,  6,  7,  8,  9],
        [ 6,  7,  8,  9, 10],
        [ 7,  8,  9, 10, 11],
        [ 8,  9, 10, 11, 12],
        [ 9, 10, 11, 12, 13]])

torch.Size([10])
tensor([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14])


### Building the Neural Network
Implementing an RNN using PyTorch's [Module class]. The initialize function creates the layers of the neural network and save them to the class. The forward propagation function uses these layers to run forward propagation and generate an output and a hidden state.

**The output of this model is the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, it output's the word scores for a single, most likely, next word.

In [11]:
import torch.nn as nn

class RNN(nn.Module):

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        # defining model layers
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)

        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()


    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        
        batch_size = nn_input.size(0)

        # embeddings and lstm_out
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden)

        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        # fully-connected layer
        out = self.fc(lstm_out)

        # reshape into (batch_size, seq_length, output_size)
        out = out.view(batch_size, -1, self.output_size)
        # get last batch
        out = out[:, -1]

        # return one batch of output word scores and the hidden state
        return out, hidden


    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''

        # initializing hidden state with zero weights, and move to GPU if available
        weight = next(self.parameters()).data

        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())

        return hidden

tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### 6. Forward and Backward Propagation
The `forward_back_prop` function computes the output of the model, calculates the loss, and updates the model parameters using backpropagation.

In [12]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """

    # move data to GPU, if available
    if(train_on_gpu):
        rnn.cuda()
        inp, target = inp.cuda(), target.cuda()

    # perform backpropagation and optimization

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in hidden])

    # zero accumulated gradients
    rnn.zero_grad()

    # getting the output from the model
    output, h = rnn(inp, h)

    # calculating the loss and perform backprop
    loss = criterion(output, target)
    loss.backward()

    # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
    nn.utils.clip_grad_norm_(rnn.parameters(), 5)
    optimizer.step()

    # returning the loss over a batch and the hidden state produced by our model
    return loss.item(), h


tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## 7. Neural Network Training
Implementing the training loop for the RNN. The `train_rnn` function iterates through the dataset for a specified number of epochs, performing forward and backward propagation for each batch of data. It prints the average loss at specified intervals to monitor the training progress.

In [13]:
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []

    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):

        # initializing hidden state
        hidden = rnn.init_hidden(batch_size)

        for batch_i, (inputs, labels) in enumerate(train_loader, 1):

            # making sure it iterates over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break

            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### 8. Hyperparameters
The hyperparameters for training the RNN are defined. These parameters includes:
- Setting `sequence_length` to the length of a sequence.
- Setting `batch_size` to the batch size.
- Setting `num_epochs` to the number of epochs to train for.
- Setting `learning_rate` to the learning rate for an Adam optimizer.
- Setting `vocab_size` to the number of uniqe tokens in our vocabulary.
- Setting `output_size` to the desired size of the output.
- Setting `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Setting `hidden_dim` to the hidden dimension of your RNN.
- Setting `n_layers` to the number of layers/cells in your RNN.
- Setting `show_every_n_batches` to the number of batches at which the neural network should print progress.

In [14]:
# Data params
# Sequence Length
sequence_length = 10  # of words in a sequence
# Batch Size
batch_size = 128

# data loader
train_loader = batch_data(int_text, sequence_length, batch_size)

In [15]:
# Training parameters
# Number of Epochs
num_epochs = 10
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 300
# Hidden Dimension
hidden_dim = 512
# Number of RNN Layers
n_layers = 2

# Showing stats for every n number of batches
show_every_n_batches = 500

## 9. Model Training
Initiating the training of the RNN model using the defined parameters.

In [16]:
# creating model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# Defining the path to the model folder within the project
model_path = os.path.join(project_dir, "model")

# Saving the model
helper.save_model(os.path.join(model_path, 'trained_rnn'), trained_rnn)
print('Model Trained and Saved')

Training for 10 epoch(s)...
Epoch:    1/10    Loss: 5.370089016914368

Epoch:    1/10    Loss: 4.798733164787293

Epoch:    1/10    Loss: 4.542001016139984

Epoch:    1/10    Loss: 4.418223347663879

Epoch:    1/10    Loss: 4.430387577533722

Epoch:    1/10    Loss: 4.464100285053253

Epoch:    1/10    Loss: 4.369395916938782

Epoch:    1/10    Loss: 4.248648833751679

Epoch:    1/10    Loss: 4.236416395664215

Epoch:    1/10    Loss: 4.164144903182984

Epoch:    1/10    Loss: 4.291093181610107

Epoch:    1/10    Loss: 4.313761456012726

Epoch:    1/10    Loss: 4.3124701805114745

Epoch:    2/10    Loss: 4.097656175863644

Epoch:    2/10    Loss: 3.909405348300934

Epoch:    2/10    Loss: 3.816537125110626

Epoch:    2/10    Loss: 3.7396456742286683

Epoch:    2/10    Loss: 3.7917556462287902

Epoch:    2/10    Loss: 3.8681172285079954

Epoch:    2/10    Loss: 3.8215636863708498

Epoch:    2/10    Loss: 3.7154536542892456

Epoch:    2/10    Loss: 3.716721570968628

Epoch:    2/10    Lo

## 10. Model Loading
The trained model and the necessary dictionaries for word-to-ID mapping are loaded from disk. This allows for generating new text using the trained RNN model without needing to retrain it.

In [17]:
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model(os.path.join(model_path, 'trained_rnn'))

/content/drive/My Drive/tv script generator project/project-generate-tv-scripts/helper.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(save_filename)


## 11. Text Generation
Processing for generating new text using the trained RNN model. The `generate` function takes a starting word ID and produces a specified length of text by predicting the next word iteratively. The generated text is then formatted to replace tokenized punctuation with actual punctuation marks.

In [18]:
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()

    # creating a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]

    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)

        # initializing the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))

        # getting the output of the rnn
        output, _ = rnn(current_seq, hidden)

        # getting the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu

        # using top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()

        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())

        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)

        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq.cpu(), -1, 1)
        current_seq[-1][-1] = word_i

    gen_sentences = ' '.join(predicted)

    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')

    # return all the sentences
    return gen_sentences

## 12. Generate a New Script
The user specifies the length of the generated script and the starting character for the dialogue. The `generate` function is called to produce a new script based on the trained model, and the generated text is printed.

In [19]:
# running this cell multiple times gets different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

jerry: enthusiastically to the stand.

chiles: so what is the penalty mans name?

george: yes, the sponge- l. yes.

george: oh, yeah, yeah.

chiles: what happened?

george: well, it was a lovely sandwich.

chiles: i think it's the most beautiful person that occurred in the parking lot lady ended four days ago.

george: i thought we were going to find out the bread.

hoyt: so what?

elaine: i don't know if it was a potential party.

hoyt: call sidra holland to the stand.

[new witness: the stand.

jerry: what is that noise?

george: yes.

elaine: oh, you know, the people have had a fat conversation, so, essentially, i think that's it.

hoyt: so what?

elaine: oh yeah, yeah.

hoyt: so you were in the bakery.

[new witness: security 1.

elaine: so, essentially, what's going on?

george: i don't know, but i have to be a lot of prostitutes science.

george: what is that noise?

jerry: i think it's going upon be swarming to california.

jerry: yeah, i think i can tell you about that. i can't

## 13. Save Generated Script

In [ ]:
# Save the script
file_path = os.path.join(project_dir, "generated_script.txt")
with open(file_path, "w") as f:
    f.write(generated_script)

print(f"File saved at: {file_path}")

The generated TV script may not be perfect, but it resemble the style of Seinfeld dialogue.